In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [12]:
#change to appropriate home folder
folder = ''

In [ ]:
train_labels = glob(folder+'train_xml/*')
drug_label_text = []
for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for section in soup.find_all('Section'):
        section_name = section['name']
        drug_label_text.append([drug_name, section_name, section.text])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'section_name', 'section_text'])
drug_label_text.to_csv(folder+'train_drug_label_text.csv', index=False)
drug_label_text.head()

100%|██████████| 101/101 [00:02<00:00, 44.62it/s]


,drug_name,section_name,section_text
0,KYPROLIS,adverse reactions,6 ADVERSE REACTIONS\n\n The following adv...
1,KYPROLIS,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...
2,MULTAQ,adverse reactions,6 ADVERSE REACTIONS\n\n The following saf...
3,MULTAQ,boxed warnings,\n\n BOXED WARNING: WARNING: INCREASED RISK...
4,MULTAQ,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...


In [ ]:
test_labels = glob(folder+'gold_xml/*')
drug_label_text = []
for label in tqdm(test_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for section in soup.find_all('Section'):
        section_name = section['name']
        drug_label_text.append([drug_name, section_name, section.text])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'section_name', 'section_text'])
drug_label_text.to_csv(folder+'test_drug_label_text.csv', index=False)
drug_label_text.head()

100%|██████████| 99/99 [00:31<00:00,  3.19it/s]


,drug_name,section_name,section_text
0,PREPOPIK,adverse reactions,6 ADVERSE REACTIONS\n\n EXCERPT: Most c...
1,PREPOPIK,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...
2,ICLUSIG,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...
3,ICLUSIG,boxed warnings,\n\n BOXED WARNING: WARNING: VASCULAR OCCLU...
4,ICLUSIG,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...


also extract the manually annotated ades.

In [37]:
train_labels = glob(folder+'train_xml/*')
drug_label_text_all = pd.DataFrame()
section_cat = []
for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    mentions = []
    for mention in soup.find_all('Mention'):
        if mention['type'] == 'AdverseReaction':
          section_name = mention['section']
          mention_str = mention['str'].lower()
          mentions.append([section_name, mention_str])
    mentions_df = pd.DataFrame(mentions, columns=['section_id', 'reaction_string'])
    ####
    drug_label_text = []
    for reaction in soup.find_all('Reaction'):
        string = reaction['str']
        for norm in reaction.find_all('Normalization'):
          try:
            meddra_str = norm['meddra_pt']
            meddra_id = norm['meddra_pt_id']
          except:
            meddra_str, meddra_id = None, None
          try:
            meddra_llt = norm['meddra_llt']
            meddra_llt_id = norm['meddra_llt_id']
          except:
            meddra_llt, meddra_llt_id = None, None
          drug_label_text.append([drug_name, string, meddra_str, meddra_id, meddra_llt, meddra_llt_id])
    drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'reaction_string', 'meddra_pt', 'meddra_pt_id', 'meddra_llt', 'meddra_llt_id'])
    ####
    drug_label_text = mentions_df.merge(drug_label_text, how='left', on='reaction_string')
    drug_label_text_all = pd.concat([drug_label_text_all, drug_label_text])
    section_cat.extend([[drug_name, i['id'], i['name']] for i in soup.find_all('Section')])

section_table = pd.DataFrame(section_cat, columns=['drug_name', 'section_id', 'section_name'])
drug_label_text_all = drug_label_text_all.merge(section_table, on=['drug_name', 'section_id'], how='left')
drug_label_text_all.to_csv(folder+'train_drug_label_text_manual_ades.csv', index=False)
drug_label_text_all.head()

100%|██████████| 101/101 [00:00<00:00, 208.96it/s]


,section_id,reaction_string,drug_name,meddra_pt,meddra_pt_id,meddra_llt,meddra_llt_id,section_name
0,S1,hypersensitivity,XEOMIN,Hypersensitivity,10020751,None,None,adverse reactions
1,S1,dysphagia,XEOMIN,Dysphagia,10013950,None,None,adverse reactions
2,S1,breathing difficulties,XEOMIN,Dyspnoea,10013968,Difficulty breathing,10012791,adverse reactions
3,S1,spread of effects from toxin,XEOMIN,None,None,None,None,adverse reactions
4,S1,dysphagia,XEOMIN,Dysphagia,10013950,None,None,adverse reactions


In [38]:
test_labels = glob(folder+'gold_xml/*')
drug_label_text_all = pd.DataFrame()
section_cat = []
for label in tqdm(test_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    mentions = []
    for mention in soup.find_all('Mention'):
        if mention['type'] == 'AdverseReaction':
          section_name = mention['section']
          mention_str = mention['str'].lower()
          mentions.append([section_name, mention_str])
    mentions_df = pd.DataFrame(mentions, columns=['section_id', 'reaction_string'])
    ####
    drug_label_text = []
    for reaction in soup.find_all('Reaction'):
        string = reaction['str']
        for norm in reaction.find_all('Normalization'):
          try:
            meddra_str = norm['meddra_pt']
            meddra_id = norm['meddra_pt_id']
          except:
            meddra_str, meddra_id = None, None
          try:
            meddra_llt = norm['meddra_llt']
            meddra_llt_id = norm['meddra_llt_id']
          except:
            meddra_llt, meddra_llt_id = None, None
          drug_label_text.append([drug_name, string, meddra_str, meddra_id, meddra_llt, meddra_llt_id])
    drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'reaction_string', 'meddra_pt', 'meddra_pt_id', 'meddra_llt', 'meddra_llt_id'])
    ####
    drug_label_text = mentions_df.merge(drug_label_text, how='left', on='reaction_string')
    drug_label_text_all = pd.concat([drug_label_text_all, drug_label_text])
    section_cat.extend([[drug_name, i['id'], i['name']] for i in soup.find_all('Section')])

section_table = pd.DataFrame(section_cat, columns=['drug_name', 'section_id', 'section_name'])
drug_label_text_all = drug_label_text_all.merge(section_table, on=['drug_name', 'section_id'], how='left')
drug_label_text_all.to_csv(folder+'test_drug_label_text_manual_ades.csv', index=False)
drug_label_text_all.head()

  0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 99/99 [00:00<00:00, 174.93it/s]


,section_id,reaction_string,drug_name,meddra_pt,meddra_pt_id,meddra_llt,meddra_llt_id,section_name
0,S1,nausea,IMPAVIDO,Nausea,10028813,None,None,adverse reactions
1,S1,vomiting,IMPAVIDO,Vomiting,10047700,None,None,adverse reactions
2,S1,diarrhea,IMPAVIDO,Diarrhoea,10012735,Diarrhea,10012727,adverse reactions
3,S1,headache,IMPAVIDO,Headache,10019211,None,None,adverse reactions
4,S1,decreased appetite,IMPAVIDO,Decreased appetite,10061428,None,None,adverse reactions
